In [ ]:
!pip install Augmentor

In [ ]:
import Augmentor
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model
from keras.layers import GlobalAveragePooling2D, Flatten, BatchNormalization, Dense, Dropout
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D

In [ ]:
from zipfile import ZipFile
with ZipFile('/content/drive/MyDrive/images.zip', 'r') as zipObj:

   zipObj.extractall('IMAGES')

In [ ]:
p = Augmentor.Pipeline("/content/IMAGES/images",output_directory="/content/output")

Initialised with 210 image(s) found.
Output directory set to /content/output.

In [ ]:
p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)

p.zoom(probability=0.3, min_factor=1.1, max_factor=1.6)

In [ ]:
p.sample(1000)

In [ ]:
import pathlib
data_dir = pathlib.Path('/content/output')
img_height=224
img_width=224

In [ ]:
import shutil
shutil.make_archive('/content/output', 'zip', root_dir='/content/drive/MyDrive/Dataset')

'/content/output.zip'

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=32
)


Found 1000 files belonging to 2 classes.
Using 800 files for training.
Found 1000 files belonging to 2 classes.
Using 200 files for validation.


In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

In [ ]:
normalization_layer = Rescaling(1./255)
train_ds_normalized = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds_normalized = val_ds.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
num_classes=2

In [ ]:
def one_hot_encoding(x, y):
    return x, tf.one_hot(y, depth=num_classes)

train_ds_encoded = train_ds_normalized.map(one_hot_encoding)
val_ds_encoded = val_ds_normalized.map(one_hot_encoding)

In [ ]:
from tensorflow.keras.applications.nasnet import NASNetLarge, NASNetMobile
from tensorflow.keras.applications.nasnet import preprocess_input
from tensorflow.keras.preprocessing import image

In [ ]:
nasnet_large_model = NASNetLarge(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
for layer in nasnet_large_model.layers:
    layer.trainable = False

In [ ]:
x = nasnet_large_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512,activation='relu')(x)
x = Dense(256,activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

In [ ]:
model_nasnet_large = Model(inputs=nasnet_large_model.input, outputs=predictions)

In [ ]:
model_nasnet_large.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model_nasnet_large.fit(
    train_ds_encoded,
    epochs=25,
    validation_data=val_ds_encoded,
    verbose=1
)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
train_label = np.concatenate([y for x, y in train_ds_encoded], axis=0)
print(train_label.shape)

(800, 2)


In [ ]:
test_label = np.concatenate([y for x, y in val_ds_normalized], axis=0)
print(test_label.shape)

(200,)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
Y_pred = model_nasnet_large.predict_generator(val_ds_normalized, 1600)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
cm=confusion_matrix(test_label, y_pred)
print(cm)
print('Classification Report')
target_names = ['Pothole','SpeedBreaker']
print(classification_report(test_label, y_pred, target_names=target_names))

In [ ]:
import seaborn as sns
sns.set(font_scale=1.0)
ax=sns.heatmap(cm, annot=True, cmap='rocket', cbar=False, linewidths=3, linecolor='r', square=True, xticklabels=target_names,yticklabels=target_names,fmt='')
sns.set(font_scale = 2.0)
ax.set_title('Seaborn Confusion Matrix with labels\n\n')
ax.set_xlabel('\nActual Values')
ax.set_ylabel('Predicted Values ')